In [2]:
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker

%run basics_sensorSignal.py
%matplotlib
matplotlib.use("Qt5Agg")

Using matplotlib backend: MacOSX


global variables

In [3]:
dcolor = dict({'pH': '#1CC49A', 'sig pH': '#4B5258', 'NH3': '#196E94', 'sig NH3': '#314945',
               'NH4': '#DCA744', 'sig NH4': '#89621A', 'TAN': '#A86349'})
ls = dict({'target': '-.', 'simulation': ':'})
fs = 11

# sensor characteristics fixed
pKa = 9.25                 # pKa of ammonia
phmin, phmax = 0, 14       # dynamic range of pH sensor
step_ph = 0.01
ph_deci = 2                # decimals for sensor sensitivity
ph_res = 1e-5              # resolution of the pH sensor 
t_plateau = 50             # int, time for one pH value in seconds

sig_max = 400              # maximal signal response at maximal NH4+ concentration in mV 
sig_bgd = 5.0              # background signal / offset in mV at 0M NH4+
tresp_pH = .01             # response time of pH sensor in seconds
E0 = (sig_bgd)/1000        # zero potential of the reference electrode
Temp = 25                  # measurement temperature in degC
tsteps = 1e-3              # time steps for pH and NH3 sensor (theory)

# electrochemical NH3 sensor
c_nh4_ppm = 230.             # concentration of ammonium ion at the pKa; in ppm
anh3_min, anh3_max = 0, 100  # dynamic range of the NH3 sensor in %
anh3_step = .01              # steps for concentration range
sigNH3_max = 0.09            # maximal signal at maximal NH3 concentration in mV (pH=1) 
sigNH3_bgd = 0.02            # background signal / offset in mV at 0M NH3
tresp_nh3 = .1               # response time of the NH3 sensor in seconds
nh3_res = 1e-9               # resolution of the NH3 sensor 
smpg_rate = 1.               # sampling rate of NH3 sensor in seconds

# --------------------------------------------------------------------------------------
# USER INPUT
# --------------------------------------------------------------------------------------
# time intervals pH fluctuations - step function
pH_steps = list(np.arange(phmin, phmax))  # steps of pH changes; linearly or randomly

random = input('Select: (1) Random pH gradient or (2) ramp function? >')
if random == '1' or random == 'random':
    np.random.shuffle(pH_steps)
elif random == '2' or random == 'ramp':
    pH_steps = pH_steps
else:
    raise ValueError('Choose between 1 and 2')

print('Targeted pH changes:', pH_steps)

Select: (1) Random pH gradient or (2) ramp function? >2
Targeted pH changes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [4]:
sensor_ph = dict({'E0': E0, 't90': tresp_pH, 'resolution': ph_res, 'time steps': tsteps, 
                  'background signal': sig_bgd, 'sensitivity': ph_deci})
sensor_nh3 = dict({'pH range': (phmin, phmax, step_ph), 'sensitivity': ph_deci, 'pKa': pKa,
                   'response time': tresp_nh3, 'resolution': nh3_res,
                   'time steps': tsteps, 'nh3 range': (anh3_min, anh3_max, anh3_step), 
                   'signal min': sigNH3_bgd, 'signal max': sigNH3_max})

para_meas = dict({'Temperature': Temp, 'Plateau time': t_plateau, 'pH steps': pH_steps,
                 'sampling rate': smpg_rate, 'GGW concentration': c_nh4_ppm})

simulate NH3 vs NH4 system

In [ ]:
df_alpha = _tan_simulation(c_nh4=c_nh4_ppm, phmin=phmin, phmax=phmax, step_ph=step_ph, 
                           ph_deci=ph_deci, pKa=pKa)

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
move_figure(xnew=50, ynew=60)

fig.canvas.set_window_title('Ammonia vs. Ammonium')
ax.set_xlabel('pH', fontsize=fs), ax.set_ylabel('alpha [%]', fontsize=fs)

ax.plot(df_alpha['nh4 %'], color=dcolor['NH4'], lw=1., label='NH$_4^+$')
ax.plot(df_alpha['nh3 %'], color=dcolor['NH3'], lw=1., label='NH$_3$')

ax.axvline(pKa, color='k', ls=':', lw=1.)
ax.axhline(0.5, color='k', ls=':', lw=1.)
ax.legend(loc='upper center', bbox_to_anchor=(1,0.9), frameon=True, fancybox=True, 
          fontsize=fs*0.7)

ax.set_xlim(-0.5, phmax*1.05)
sns.despine()
plt.tight_layout()
plt.show()

simulate experiment -  pH changes over time

In [ ]:
#pH sensor response - electrochemical pH sensor calibration
#create time range
l = [[ph_i] * t_plateau for ph_i in pH_steps]
ls_pH = l[0]
lspH = [ls_pH.extend(l_i) for l_i in l[1:]]

#finalize target pH and concentrations
df_ph = pd.DataFrame(ls_pH, columns=['pH'])
df_pH = _alpha4ph(df_pH=df_ph, df_alpha=df_alpha)
para_meas['pH steps'] = pH_steps

# compare: https://www.vernier.com/2018/04/06/the-theory-behind-ph-measurements/
dfSig_calib = pd.DataFrame(Nernst_equation(ls_ph=np.array(pH_steps), T=Temp, E0=E0),
                           index=pH_steps, columns=['signal / mV'])
dfSig_calib.index.name = 'pH'

In [ ]:
# calibration plot pH sensor
fig, ax = plt.subplots(figsize=(5,3))
move_figure(xnew=50, ynew=450)

fig.canvas.set_window_title('Calibration pH sensor (electrochemical)')
ax.set_xlabel('pH', fontsize=fs), ax.set_ylabel('Potential / mV', fontsize=fs)

ax.plot(dfSig_calib, lw=1., color='k')

sns.despine()
plt.tight_layout()

In [ ]:
# pH changes -> target potential
dfSig_steps = pd.DataFrame(Nernst_equation(ls_ph=np.array(df_pH['pH'].to_numpy()), T=Temp, 
                                           E0=E0), index=df_pH['pH'].index, 
                           columns=['target potential / mV'])
dfpH_target = pd.concat([df_pH['pH'], dfSig_steps], axis=1)

# include sensor response
sens_response = _pHsensor_response(t_plateau=t_plateau, pH_plateau=pH_steps, ph_res=ph_res,
                                   dfpH_target=dfpH_target, t90_pH=tresp_pH, step=tsteps,
                                   sig_bgd=sig_bgd)

# recalculation of pH values
dfph_re = _potential2pH(sens_response=sens_response, ph_deci=ph_deci, E0=E0, T=Temp)

In [ ]:
# final results pH sensor
figR, axR = plt.subplots(figsize=(5.8,4))
move_figure(700, 70)
figR.canvas.set_window_title('pH sensor response')
axT = axR.twinx()
axR.set_xlabel('Time / s', fontsize=fs)
axR.set_ylabel('Potential / mV', color=dcolor['NH3'], fontsize=fs)
axT.set_ylabel('pH', fontsize=fs, color=dcolor['pH'])

# target potential
lns1 = axR.plot(dfpH_target['target potential / mV'], color='k', ls=ls['target'], lw=1., 
                label='target potential / mV')
# sensor response
lns2 = axR.plot(sens_response, color=dcolor['NH3'], lw=1., label='sensor response / mV')

# re-calculated pH
lns4 = axT.plot(dfph_re, color=dcolor['pH'], lw=1., label='recalc. pH')
lns3 = axT.plot(dfpH_target['pH'], lw=1., ls=ls['target'], color='gray', label='target pH')

axT.set_ylim(-0.5, phmax*1.05)

# combine legend
lns = lns1+lns2+lns3+lns4
labs = [l.get_label() for l in lns]
axR.legend(lns, labs, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 1.1), 
           fontsize=fs*0.7)
plt.tight_layout()

NH3 sensor

In [ ]:
# NH3 sensor response
# lin.calibration of an electrochemical sensors at a specific pH; done for different pH
conc_nh3 = np.arange(anh3_min, anh3_max, step=anh3_step)

# linear regression for all pH values
para_nh3 = _calibration_nh3(anh3_min=anh3_min, anh3_max=anh3_max, anh3_step=anh3_step, 
                            sigNH3_bgd=sigNH3_bgd, sigNH3_max=sigNH3_max)

In [ ]:
fig3, ax3 = plt.subplots(figsize=(5, 3))
move_figure(50, 500)

fig3.canvas.set_window_title('Calibration NH3 sensor (electrochemical)')
ax3.set_xlabel('alpha(NH$_3$) / %', fontsize=fs*0.9)
ax3.set_ylabel('Sensor signal / mV', fontsize=fs*0.9)

ax3.plot(conc_nh3, para_nh3[0]*conc_nh3 + para_nh3[1], lw=1., color='k')

sns.despine()
plt.tight_layout()

In [ ]:
# target NH3 signal for NH3 and NH4+ according to the measured pH
# specific target concentration (NH3) over time in percentages
nh3_target, nh4_target, dfalpha_target = _alpha_vs_time(df_alpha=df_alpha, 
                                                        dfpH_target=dfpH_target)

# =====================================================================================
#[USER INPUT]
# conversion of alpha_NH3 to an electrochemical signal in mV depending on pH (given)
cnh3_target = [c_nh4_ppm]*len(dfalpha_target.index) # concentration of NH3 over time
cnh4_target = cnh3_target

# =====================================================================================
[dfconc_target, 
 dfpot_target] = conv2potent_nh3(dfalpha_target=dfalpha_target, c_nh3=cnh3_target, 
                                 c_nh4=cnh4_target, dfpH_target=dfpH_target, 
                                 para_nh3=para_nh3)

In [ ]:
# include sensor response - double checked. NH3 starting with ideal situation
sensNH3_resp = _nh3sensor_response(t_plateau=t_plateau, pH_plateau=pH_steps, step=tsteps,
                                   dfph_re=dfph_re, t90_nh3=tresp_nh3, nh3_res=nh3_res, 
                                   dfpot_target=dfpot_target)

# recalculation of NH3 concentrations while considering the pH(recalc.)
dfnh3_calc = _potent2nh3(sensNH3_response=sensNH3_resp, para_nh3=para_nh3)

# recalculation of NH3 concentrations while considering the pH(recalc.)
dfnh3_ = _potent2nh3(sensNH3_response=sensNH3_resp, para_nh3=para_nh3)
dfnh3_calc = pd.concat([dfnh3_, dfph_re], axis=1).dropna()

# get NH4 concentration via Henderson-Hasselbalch
dfnh4_calc = pd.DataFrame(henderson_nh4(pKa=pKa, pH=dfnh3_calc['pH recalc.'], 
                                        c_nh3=dfnh3_calc['nh3 / ppm']), 
                          columns=['nh4 / ppm'], index=dfnh3_calc.index)

# include sampling rate
x_smg = np.arange(dfnh3_calc.index[0], dfnh3_calc.index[-1], para_meas['sampling rate'])
x_smg = [i.round(2) for i in x_smg]

df_record = pd.concat([dfnh3_calc, dfnh4_calc], axis=1)
tnew = [round(i, 2) for i in df_record.index]
df_record.index = tnew
df_record = df_record.loc[x_smg]

# TAN = NH3 + NH4+
df_tan = pd.DataFrame(df_record[['nh3 / ppm', 'nh4 / ppm']].sum(axis=1), columns=['TAN'])
df_tan_target = pd.DataFrame(dfconc_target[['nh3 / ppm', 'nh4 / ppm']].sum(axis=1),
                             columns=['TAN'])

In [ ]:
fig4 = plt.figure(figsize=(6, 4.5))
move_figure(690, 350)
fig4.canvas.set_window_title('NH3 / NH4+ simulation')

gs = GridSpec(nrows=3, ncols=1)
ax4 = fig4.add_subplot(gs[:2, 0])
ax4_ = fig4.add_subplot(gs[2, 0], sharex=ax4)
ax4_ph = ax4_.twinx()
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)

ax4_ph.set_xlabel('Time / s'), 
ax4_.set_ylabel('TAN / ppm', color=dcolor['TAN']), ax4_ph.set_ylabel('pH', color='gray')
ax4.set_ylabel('NH$_4^+$ & NH$_3$ / ppm')
ax4_ph.set_ylim(-0.5, phmax*1.05)

# top plot
a, = ax4.plot(df_record['nh3 / ppm'], lw=1., color=dcolor['NH3'], label='NH$_3$')
b, = ax4.plot(df_record['nh4 / ppm'], lw=1., color=dcolor['NH4'],  label='NH$_4^+$')
ax4.legend(frameon=True, fancybox=True, loc=0, fontsize=fs*0.7)
c, = ax4.plot(dfconc_target['nh3 / ppm'], lw=1., ls=ls['target'], color='k')
d, = ax4.plot(dfconc_target['nh4 / ppm'], lw=1., ls=ls['target'], color='gray')

# bottom plot
e, = ax4_.plot(df_tan_target, lw=1., ls=ls['target'], color='k')
f, = ax4_.plot(df_tan, lw=1., color=dcolor['TAN'])
g, = ax4_ph.plot(dfph_re, lw=1., ls='-.', color='gray')

plt.tight_layout()

save / load data

In [39]:
[dic_target, dic_sens_calib, dsens_record, dic_figures, 
 df_tan] = tan_simulation(sensor_ph, para_meas, sensor_nh3, plot=False)

In [154]:
out = save_report(para_meas, sensor_ph, sensor_nh3, dsens_record, dtarget=dic_target)
out

,0,1,2,3,4,5,6,7
general,parameter,values,NaN,NaN,NaN,NaN,NaN,NaN
general,Plateau time,50,NaN,NaN,NaN,NaN,NaN,NaN
general,pH steps,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",NaN,NaN,NaN,NaN,NaN,NaN
general,sampling rate,1.0,NaN,NaN,NaN,NaN,NaN,NaN
general,GGW concentration,230.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
695,230.0,230.0,229.959107,229.959107,0.040893,0.040893,13.0,13.0
696,230.0,230.0,229.959107,229.959107,0.040893,0.040893,13.0,13.0
697,230.0,230.0,229.959107,229.959107,0.040893,0.040893,13.0,13.0
698,230.0,230.0,229.959107,229.959107,0.040893,0.040893,13.0,13.0


In [175]:
df_general, df_ph, df_nh3 = load_data(file='results/test1.txt')